### Historic Event Pipeline Runs

#### This notebook will kick off pipelines for specified configurations for a given date range & reference times. At present, past events can be run in the TI pipelines, but this may not be the case at a later date, depending on what's in the development queue. You may need to create a seperate set of pipeline resources in AWS, if so (see step 1 below).

##### Last Update: 2024-01-25

I currently employ the following workflow to generate past event data.
1. [Optional] Use terraform to create a custom set of vpp lambda functions and step functions (currently `hv-vpp-dev-historical-run-{function name}` from the desired HydroVIS repo branch / version, without any SNS triggers on the initialize pipeline function (so it will only run on-demmand).
2. Setup individual single-reference-time test events for the configurations I intend  to run in the initialize_pipeline lambda function created in the last step and throroughly test (first with the `invoke_step_function` arg set to false, then true when ready) - Pay particular attention to the table names and sql_replace dictionary, to ensure that archive tables are being used througout the entire pipeline. Looking at these tables in pg_admin is also a good idea to make sure everything looks good.
Here is an example of a test event to try in lambda:

{
  "configuration": "analysis_assim",
  "bucket": "nws-shared-data-226711853580-us-east-1",
  "reference_time": "2024-01-17 12:00:00",
  "invoke_step_function": false
}

3. Use the Source\Visualizations\aws_loosa\utils\viz_cache_csvs_to_clipped_geospatial.py script to retrieve and convert the test data into gpkg, and examine in Arc Pro. Make sure you use "ref" as a search criteria, which will ensure you get only past event data files.
4. When full step function execution and outputs have been verified, input the new initialize pipeline lambda arn into this notebook, and use it to batch run pipelines.
5. Use the Source\Visualizations\aws_loosa\utils\viz_cache_csvs_to_clipped_geospatial.py to fetch, convert, and clip to state (if desired) generated data for users.
6. Upload files to Google Drive for sharing.

A few caveats/warnings you should consider before using this notebook:
- We currently do not regurlaly test past/historic event functionality with regular piplines updates, so pipelines may not run correctly when using this functionality without significant testing and tweaking, if there have been major development updates since the last refresh of this process.
- Creating a custom version of the Viz RDS database instance would be required to truly duplicate the environment of a given HydroVIS version, which is not tested/supported with our current db_dump strategy (although may be possible with some manual work).
- If using the default viz RDS instance, keep in mind that derived data will reflect what's currently in the TI database, not what's in Production.
- If using the defualt viz RDS instance, also keep in mind that historic pipelines ran here will be competing for database resources with the automatic operational viz pipelines, which may cause issues. I've sometimes disabled those triggers to initialize_pipeline temporarily when running large batches.
- All csv files archived to S3 will include a reference time prefix to denote they came from a past event pipeline (contrary to regularly archived files, which just contain the service name). That said, if FIM is filtered to specific states, that information is not presently included in the csv file names, which could cause issues if filtered and non-filtered versions are desired.
- In the past, Chip would run RnR manually to create outputs that we would run through the pipelines in this workflow. That process needs to be updated based on Shawn Crawleys late-2023 updates to RnR workflows.

In [1]:
from datetime import date
from helper_functions.shared_functions import run_pipelines

In [2]:
# NWM 3.0 Testing Runs for Katherine - 2024 0125
events = [
    # {"start_date": date(2023, 12, 21), "end_date": date(2023, 12, 21), "reference_times": ["09:00:00", "10:00:00"]},
    {"start_date": date(2023, 12, 17), "end_date": date(2023, 12, 18), "reference_times": ["18:00:00"]},
    # {"start_date": date(2023, 12, 17), "end_date": date(2023, 12, 17), "reference_times": ["13:00:00", "14:00:00"]},
    # {"start_date": date(2023, 12, 5), "end_date": date(2023, 12, 5), "reference_times": ["14:00:00", "15:00:00"]},
    # {"start_date": date(2023, 12, 2), "end_date": date(2023, 12, 2), "reference_times": ["13:00:00", "14:00:00"]},
    # {"start_date": date(2023, 10, 28), "end_date": date(2023, 10, 28), "reference_times": ["09:00:00", "10:00:00"]},
    # {"start_date": date(2023, 10, 24), "end_date": date(2023, 10, 24), "reference_times": ["16:00:00", "17:00:00"]},
    # {"start_date": date(2023, 10, 4), "end_date": date(2023, 10, 4), "reference_times": ["21:00:00", "22:00:00"]},
    # {"start_date": date(2023, 10, 3), "end_date": date(2023, 10, 3), "reference_times": ["22:00:00", "23:00:00"]}
]
for event in events:
    start_date = event['start_date']
    end_date = event['end_date']
    reference_hours = event['reference_times']
    configurations = [{"analysis_assim": "nws-shared-data-226711853580-us-east-1"}]
    interval_minutes = 10 # specify the number of minutes to wait between pipeline invokations - this is important to get right to not overwhelm the database
    initialize_pipeline_arn = "arn:aws:lambda:us-east-1:526904826677:function:hv-vpp-ti-viz-initialize-pipeline"
    run_pipelines(start_date, end_date, reference_hours, configurations, initialize_pipeline_arn, interval_minutes=interval_minutes)

Invoked viz_initialize_pipeline function with payload: {'configuration': 'analysis_assim', 'bucket': 'nws-shared-data-226711853580-us-east-1', 'reference_time': '2023-12-17 18:00:00', 'states_to_run_fim': None, 'skip_fim': False}.
Invoked viz_initialize_pipeline function with payload: {'configuration': 'analysis_assim', 'bucket': 'nws-shared-data-226711853580-us-east-1', 'reference_time': '2023-12-18 18:00:00', 'states_to_run_fim': None, 'skip_fim': False}.
